In [38]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("Telecom_customer churn (100000).csv")

In [39]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)

change_mou=num_df['change_mou']
change_rev=num_df['change_rev']
num_df=num_df.drop(['change_mou'], axis=1)
num_df=num_df.drop(['change_rev'], axis=1)

num_df=num_df.drop(['Customer_ID'], axis=1)

churn=num_df['churn']
num_df=num_df.drop(['churn'], axis=1)


num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df

X=result_df_op
y=churn

In [40]:
#pip install mlencoders
from mlencoders.weight_of_evidence_encoder import WeightOfEvidenceEncoder

enc = WeightOfEvidenceEncoder(cols=X.columns)
X_encoded = enc.fit_transform(X, y)
X_encoded.head()

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,drop_vce_Mean,...,infobase,HHstatin,dwllsize,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd
0,0.000000,0.556517,0.017520,-0.022460,-0.015257,-0.014783,-0.033784,0.018887,0.014593,-0.011647,...,-0.025602,-0.036986,-0.045587,-0.028551,-0.002808,-0.000904,0.002464,0.00339,-0.001641,-0.021631
1,0.017520,-0.164801,0.000000,-0.022460,0.045300,-0.107643,-0.112533,0.018887,0.014593,-0.060797,...,-0.025602,-0.036986,-0.045587,-0.400201,-0.002808,-0.000904,0.002464,0.00339,-0.001641,-0.021631
2,-0.015816,0.751490,0.094113,0.047296,-0.015257,-0.014783,-0.033784,0.018887,0.014593,-0.028216,...,-0.025602,-0.036986,-0.045587,-0.028551,-0.002808,0.043076,0.002464,0.00339,-0.001641,-0.021631
3,-1.081092,0.199842,0.017520,0.047296,-0.015257,-0.014783,-0.033784,0.018887,0.014593,0.175511,...,-0.025602,-0.036986,0.052330,0.018434,0.095326,-0.000904,0.002464,0.00339,-0.001641,-0.021631
4,-0.270162,-0.339154,-0.136630,0.047296,-0.015257,-0.014783,-0.033784,0.018887,0.014593,0.063701,...,-0.025602,-0.036986,-0.037101,0.066336,-0.002808,-0.000904,0.002464,0.00339,-0.001641,-0.021631


In [41]:
X_encoded[X_encoded < 0]=0
X_encoded.head()

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,drop_vce_Mean,...,infobase,HHstatin,dwllsize,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd
0,0.00000,0.556517,0.017520,0.000000,0.0000,0.0,0.0,0.018887,0.014593,0.000000,...,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.002464,0.00339,0.0,0.0
1,0.01752,0.000000,0.000000,0.000000,0.0453,0.0,0.0,0.018887,0.014593,0.000000,...,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.002464,0.00339,0.0,0.0
2,0.00000,0.751490,0.094113,0.047296,0.0000,0.0,0.0,0.018887,0.014593,0.000000,...,0.0,0.0,0.00000,0.000000,0.000000,0.043076,0.002464,0.00339,0.0,0.0
3,0.00000,0.199842,0.017520,0.047296,0.0000,0.0,0.0,0.018887,0.014593,0.175511,...,0.0,0.0,0.05233,0.018434,0.095326,0.000000,0.002464,0.00339,0.0,0.0
4,0.00000,0.000000,0.000000,0.047296,0.0000,0.0,0.0,0.018887,0.014593,0.063701,...,0.0,0.0,0.00000,0.066336,0.000000,0.000000,0.002464,0.00339,0.0,0.0


In [42]:
X.shape

(100000, 96)

In [45]:
X_train=X_encoded
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=90).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
53,adjmou,4469.047321
50,totmou,4321.946233
57,avgqty,3940.782983
55,avgrev,3117.288810
30,mouowylisv_Mean,2820.734757
0,rev_Mean,2749.843626
54,adjqty,2368.620739
49,totcalls,2181.358870
1,mou_Mean,2116.792847
56,avgmou,1829.590695


In [46]:
y_train=y
x_train_chi = select_feature.transform(X_train)
x_train_chi.shape

(100000, 90)

In [43]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X_train,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

43529 6909 22937 26625
pod:  0.5372059238933053
pof:  0.13698005472064714
AUC:  0.7001129345863291


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 30 folds for each of 100 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   17.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   58.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed: 14.6min finished


{'var_smoothing': 1e-09}
GaussianNB(priors=None, var_smoothing=1e-09)
0.5169666666666667


In [32]:
{'var_smoothing': 1e-09}
GaussianNB(priors=None, var_smoothing=1e-09)
0.51704


26710 23728 21590 27972
pod:  0.5643840038739357
pof:  0.4704389547563345
AUC:  0.5469725245588006


In [41]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=1e-09)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

43513 6925 22451 27111
pod:  0.5470118235745127
pof:  0.1372972758634363
AUC:  0.7048572738555382
accuracy:  0.70624


In [1]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

    

accuracy:  0.54812


In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 16.8min finished


{'C': 5, 'multi_class': 'ovr', 'penalty': 'l1', 'solver': 'liblinear'}
LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
0.79606


In [40]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


41123 9315 11080 38482
pod:  0.776441628667124
pof:  0.18468218406756812
AUC:  0.7958797222997779
accuracy:  0.79605


In [ ]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=3, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 207.5min finished


{'metric': 'euclidean', 'n_neighbors': 11, 'weights': 'distance'}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='distance')
0.72368


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

# Specify parameters

Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gammas = [0.001, 0.01, 0.1, 1, 1e-3, 1e-4]
param_grid = {'C': Cs, 'gamma' : gammas}

tuned_parameters = [{'kernel': ['rbf', 'linear'], 
                     'gamma': [0.001, 0.01, 0.1, 1, 1e-3, 1e-4], 
                     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} ]

grid = GridSearchCV(svm.SVC(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 42 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 410.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 1993.6min


In [17]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='distance' )  
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


45856 4582 22368 27194
pod:  0.5486864936846778
pof:  0.09084420476624767
AUC:  0.7289211444592151
accuracy:  0.7305


In [18]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=19)  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


46533 3905 23033 26529
pod:  0.5352689560550422
pof:  0.07742178516198105
AUC:  0.7289235854465306
accuracy:  0.73062


In [11]:
for 19 
46533 3905 23033 26529
pod:  0.5352689560550422
pof:  0.07742178516198105
AUC:  0.7289235854465306
accuracy:  0.73062
    
    
    45856 4582 22369 27193
pod:  0.5486663169363626
pof:  0.09084420476624767
AUC:  0.7289110560850575
accuracy:  0.73049


{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 44.0min
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 82.0min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed: 113.1min finished


{'bootstrap': True, 'max_depth': 80, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 12, 'n_estimators': 1000}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=12,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.78775


In [33]:
print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

{'bootstrap': True, 'max_depth': 80, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 12, 'n_estimators': 1000}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=12,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.78775


In [34]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 12, n_estimators= 1000)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

39476 10962 10113 39449
pod:  0.7959525442879626
pof:  0.21733613545342798
AUC:  0.7893082044172672
accuracy:  0.78925


In [ ]:
{'bootstrap': True, 'max_depth': 100, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 1000}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
========================
39671 10767 10370 39192
pod:  0.7907671199709455
pof:  0.213470002775685
AUC:  0.7886485585976302
accuracy:  0.78863

In [22]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
#classifier =RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
classifier =RandomForestClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


40436 10002 16269 33293
pod:  0.6717444816593358
pof:  0.19830286688607796
AUC:  0.7367208073866289


In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 


#param_grid={
#    'criterion': ['gini', 'entropy'],
#    'splitter': ['best', 'random'],
#    'min_samples_split' : range(10,500,20),
#    'max_depth': [1, 2, 3, 4, 5, 20],
#    'max_features': [1, 2, 3, 4],
#    'max_leaf_nodes': list(range(2, 100)), 
#    'min_samples_split': [2, 3, 4]
#} 

param_grid={
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
} 

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   16.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   16.7s finished


{'criterion': 'entropy', 'splitter': 'best'}
DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
0.67634


In [34]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

34866 15572 16514 33048
pod:  0.6668011783221016
pof:  0.3087354772195567
AUC:  0.6790328505512724
accuracy:  0.67914


In [29]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

34767 15671 16767 32795
pod:  0.6616964609983456
pof:  0.3106982830405646
AUC:  0.6754990889788904


In [50]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    
    }

grid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=10, n_jobs=-1)

grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)


{}
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
0.79167


In [51]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

40877 9561 11272 38290
pod:  0.7725676929905977
pof:  0.18955945913795155
AUC:  0.7915041169263232
accuracy:  0.79167


In [36]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

40877 9561 11272 38290
pod:  0.7725676929905977
pof:  0.18955945913795155
AUC:  0.7915041169263232
accuracy:  0.79167


In [52]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 


Using TensorFlow backend.


Train on 71999 samples, validate on 18000 samples
Epoch 1/30
71999/71999 [==============================] - 8s 111us/step - loss: 0.4494 - accuracy: 0.7878 - val_loss: 0.4044 - val_accuracy: 0.8203
Epoch 2/30
71999/71999 [==============================] - 7s 100us/step - loss: 0.4284 - accuracy: 0.8005 - val_loss: 0.3944 - val_accuracy: 0.8284
Epoch 3/30
71999/71999 [==============================] - 8s 107us/step - loss: 0.4239 - accuracy: 0.8029 - val_loss: 0.3880 - val_accuracy: 0.8322
Epoch 4/30
71999/71999 [==============================] - 7s 102us/step - loss: 0.4204 - accuracy: 0.8061 - val_loss: 0.3971 - val_accuracy: 0.8291
Epoch 5/30
71999/71999 [==============================] - 8s 106us/step - loss: 0.4188 - accuracy: 0.8060 - val_loss: 0.3865 - val_accuracy: 0.8317
Epoch 6/30
71999/71999 [==============================] - 8s 114us/step - loss: 0.4176 - accuracy: 0.8062 - val_loss: 0.3919 - val_accuracy: 0.8267
Epoch 7/30
71999/71999 [==============================] - 7s 1

In [54]:
#RNN
def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 71999 samples, validate on 18000 samples
Epoch 1/5
71999/71999 [==============================] - 51s 704us/step - loss: 0.6187 - acc: 0.6564 - val_loss: 0.5793 - val_acc: 0.7198
Epoch 2/5
71999/71999 [==============================] - 46s 636us/step - loss: 0.5814 - acc: 0.7124 - val_loss: 0.5842 - val_acc: 0.7198
Epoch 3/5
71999/71999 [==============================] - 46s 632us/step - loss: 0.5789 - acc: 0.7134 - val_loss: 0.5654 - val_acc: 0.7198
Epoch 4/5
71999/71999 [==============================] - 46s 633us/step - loss: 0.5810 - acc: 0.7135 - val_loss: 0.8698 - val_acc: 0.4088
Epoch 5/5
71999/71999 [==============================] - 46s 643us/step - loss: 0.5796 - acc: 0.7129 - val_loss: 0.5845 - val_acc: 0.7198
y22_pred:  [[0.]
 [1.]
 [0.]
 ...
 [0.]
 [1.]
 [1.]]
pod 1st:  0.7236231591688521


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 71999 samples, validate on 18000 samples
Epoch 1/5
71999/71999 [==============================] - 51s 709us/step - loss: 0.6155 - acc: 0.6605 - val_loss: 0.5771 - val_acc: 0.7198
Epoch 2/5
71999/71999 [==============================] - 48s 662us/step - loss: 0.5816 - acc: 0.7133 - val_loss: 0.5778 - val_acc: 0.7171
Epoch 3/5
71999/71999 [==============================] - 47s 651us/step - loss: 0.5747 - acc: 0.7145 - val_loss: 0.5627 - val_acc: 0.7198
Epoch 4/5
71999/71999 [==============================] - 49s 674us/step - loss: 0.5741 - acc: 0.7136 - val_loss: 0.5693 - val_acc: 0.7198
Epoch 5/5
71999/71999 [==============================] - 48s 661us/step - loss: 0.5818 - acc: 0.7098 - val_loss: 0.6086 - val_acc: 0.7198
y22_pred:  [[0.]
 [0.]
 [1.]
 ...
 [1.]
 [0.]
 [0.]]
pod 1st:  0.7252370385313698


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

Train on 72000 samples, validate on 18000 samples
Epoch 1/5
72000/72000 [==============================] - 48s 661us/step - loss: 0.6924 - acc: 0.5131 - val_loss: 0.6058 - val_acc: 0.7069
Epoch 2/5
72000/72000 [==============================] - 46s 645us/step - loss: 0.5830 - acc: 0.7096 - val_loss: 0.5740 - val_acc: 0.7224
Epoch 3/5
72000/72000 [==============================] - 47s 649us/step - loss: 0.5856 - acc: 0.7133 - val_loss: 0.5761 - val_acc: 0.7224
Epoch 4/5
72000/72000 [==============================] - 47s 659us/step - loss: 0.5856 - acc: 0.7153 - val_loss: 0.5806 - val_acc: 0.7224
Epoch 5/5
72000/72000 [==============================] - 47s 649us/step - loss: 0.5844 - acc: 0.7159 - val_loss: 0.5754 - val_acc: 0.7224
y22_pred:  [[1.]
 [1.]
 [1.]
 ...
 [0.]
 [1.]
 [0.]]
pod 1st:  0.6834140435835351


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 72000 samples, validate on 18001 samples
Epoch 1/5
72000/72000 [==============================] - 47s 654us/step - loss: 0.6205 - acc: 0.6557 - val_loss: 0.6555 - val_acc: 0.7177
Epoch 2/5
72000/72000 [==============================] - 46s 646us/step - loss: 0.5816 - acc: 0.7074 - val_loss: 0.6013 - val_acc: 0.7177
Epoch 3/5
72000/72000 [==============================] - 49s 683us/step - loss: 0.5781 - acc: 0.7109 - val_loss: 0.5726 - val_acc: 0.7177
Epoch 4/5
72000/72000 [==============================] - 47s 650us/step - loss: 0.5765 - acc: 0.7127 - val_loss: 0.5686 - val_acc: 0.7177
Epoch 5/5
72000/72000 [==============================] - 47s 653us/step - loss: 0.5768 - acc: 0.7106 - val_loss: 0.5650 - val_acc: 0.7177
y22_pred:  [[1.]
 [0.]
 [1.]
 ...
 [0.]
 [1.]
 [0.]]
pod 1st:  0.7374899112187248


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 72000 samples, validate on 18001 samples
Epoch 1/5
72000/72000 [==============================] - 49s 679us/step - loss: 0.6145 - acc: 0.6615 - val_loss: 0.5607 - val_acc: 0.7198
Epoch 2/5
72000/72000 [==============================] - 46s 637us/step - loss: 0.5817 - acc: 0.7098 - val_loss: 0.5727 - val_acc: 0.7239
Epoch 3/5
72000/72000 [==============================] - 48s 665us/step - loss: 0.5788 - acc: 0.7117 - val_loss: 0.5622 - val_acc: 0.7239
Epoch 4/5
72000/72000 [==============================] - 47s 654us/step - loss: 0.5765 - acc: 0.7122 - val_loss: 0.5766 - val_acc: 0.7239
Epoch 5/5
72000/72000 [==============================] - 47s 657us/step - loss: 0.5767 - acc: 0.7122 - val_loss: 0.5707 - val_acc: 0.7239
y22_pred:  [[0.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
pod 1st:  0.733454398708636
pod:  [0.7236231591688521, 0.7252370385313698, 0.7058111380145279, 0.7249798224374495, 0.7215496368038741, 0.7193301049233253, 0.6190476190476191, 0.6834140435835351, 0.7374899112